# Return Electron Output Values from Lattices

To avoid causing graph construction issues and being scolded by the Covalent server post-processor, ensure that the value returned by a lattice is the output of an electron.

Note: Many best practices with Covalent boil down to "use the decorators." Putting as much of the working code as possible inside Covalent's decorators (electrons and lattices) enables the Covalent server to manage execution as intended.

This step in the workflow execution cycle is mostly to ensure that the correct type of return value is returned from the lattice. This step is quite resource consuming and since we'd like to keep the lattice as general as possible, for now it is run for every kind of workflow. But this can be avoided if in your workflow, you return an electron from the lattice instead of a custom object. Method (2) of writing a workflow is more preferrable than Method (1). Sometimes, certain return types will not even be supported, which is all the more reason to prefer "electron"izing wherever needed.
 
## Context

Covalent allows you to put business logic in a lattice outside of an electron, but that doesn't mean you should do so. Violating this practice puts results outside of Covalent's ability to manage execution. Following this practice ensures that your lattices, at least in this respect, will work with future versions of Covalent.

## Best Practice

Keep computations inside electrons. Use lattices to execute sequences of electrons, not to perform computations.

## Example

Contrast the two examples below.

### Not Recommended

This example demonstrates an incorrect approach. Notice all the computation that occurs in the lattice: a list of random samples is created, then returned as a numpy array. However, the list cannot be created when the lattice is run since `res`, the second parameter to the `random.sample()` method, is an unexecuted electron. As a result, the `random.sample()` method fails.


In [9]:
import covalent as ct
import numpy as np
import random

# Technique 1:

@ct.electron
def task_1(x):
    return x * 2

@ct.lattice
def workflow(a):
    res = task_1(a)
    res_list = random.sample(range(10, 30), res) # this will fail at graph construction time since `res` is still an Electron
    return np.array(res_list)

# Uncomment the two following statements to demonstrate 
id = ct.dispatch(workflow)(1)
result = ct.get_result(id, wait=True)

TypeError: '<=' not supported between instances of 'int' and 'Electron'

### Improved

In contrast, the  following code properly contains the construction of `res_list` in an electron, `task_2_new`.


In [10]:
import covalent as ct
import numpy as np
import random

# Technique 2:

@ct.electron
def task_1(x):
    return x * 2

@ct.electron
def task_2_new(x):
    res_list = random.sample(range(10, 30), x)
    return np.array(res_list)

@ct.lattice
def workflow_2(a):
    res_1 = task_1(a)
    return task_2_new(res_1)

id = ct.dispatch(workflow_2)(1)
result = ct.get_result(id, wait=True)

print(result)


Lattice Result
status: COMPLETED
result: [24 29]
input args: ['1']
input kwargs: {}
error: None

start_time: 2023-02-07 20:21:54.663980
end_time: 2023-02-07 20:21:54.793349

results_dir: /Users/mini-me/agnostiq/covalent/doc/source/developer/patterns/results
dispatch_id: 0dc7657c-5fd5-41cb-9b5c-e0ec20140724

Node Outputs
------------
task_1(0): 2
:parameter:1(1): 1
task_2_new(2): [24 29]



## Keywords

Typically glossary terms that might be useful in looking up the technique.

## Benefits

Typical benefits that might appear here:
- Performance
- Simplicity
- Maintainability
- Resource (CPU, memory) conservation
- Efficiency
- Workaround (to a problem with Python or with Covalent)

## See Also

Other Best Practices or How-to. Other documentation (Concepts, API Reference) if especially applicable.